In [ ]:
!pip install scikit-learn==1.2.0

In [ ]:
import pandas as pd
df_list = [pd.read_csv('/content/drive/MyDrive/SOC/Data/AgEvidence/cover crops/cover_crops0.csv')]
df = pd.concat(df_list, axis = 0)
df

In [ ]:
df.rv = df.rv.str.lower()

In [ ]:
df = df.query('rv.str.contains("soil") and rv.str.contains("organic") and rv.str.contains("carbon") and not rv.str.contains("poc") and not rv.str.contains("particulate")', engine='python')

In [ ]:
df.rv.value_counts()

In [ ]:
df.rvUnits.value_counts()

In [ ]:
# change all g/kg variants to g/kg
units = list(df.rvUnits)
for i in range(len(units)):
    if "/kg" in units[i]:
        units[i] = "g/kg"

# df.rvUnits = units
# df.rvUnits.value_counts()

In [ ]:
# change to percent
control_values = list(df.controlValue)
for i in range(len(units)):
    # g/100 g and % are already in percent
    if units[i] == "g/kg":
        control_values[i] *= 0.1

# df.controlValue = control_values

In [ ]:
df.columns

In [ ]:
df["Number of Species"].value_counts()

In [ ]:
feature_cols = [
    "rvUnits",
    "controlValue",
    "Type of Species",
    "Number of Species",
    "sampleDepth",
    "studyLength",
    "startYear",
    "norm"
]

In [ ]:
X = df[feature_cols]
y = df['percentChange']

In [ ]:
X.isnull().sum()

In [ ]:
X.sampleDepth.value_counts()

In [ ]:
X

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
to_encode = ["Type of Species", "Number of Species", "sampleDepth", "rvUnits"]

X[to_encode] = encoder.fit_transform(X[to_encode])

In [ ]:
X

In [ ]:
X.fillna(0, inplace = True)
X.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

# Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# XGBoost

In [ ]:
from xgboost import XGBRegressor

X_train_xgb, X_val, y_train_xgb, y_val = train_test_split(X_train, y_train)

xgb = XGBRegressor(n_estimators=1500, learning_rate=0.01, eval_metric="rmse", early_stopping_rounds=10)
xgb.fit(X_train, y_train, verbose =100)

# SVM

In [ ]:
from sklearn.svm import SVR

svr = SVR(C=1.0, epsilon=0.2)
svr.fit(X_train, y_train)

# LGBM

In [ ]:
import lightgbm

hyper_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.005,
    'verbose': -1,
    'n_estimators': 1000,
    'random_state' : 0
}

lgbm = lightgbm.LGBMRegressor(**hyper_params)

lgbm.fit(X_train_xgb,
         y_train_xgb,
         eval_set = [(X_val, y_val)],
         callbacks = [lightgbm.early_stopping(stopping_rounds = 20)],
         verbose = 100
)

# RMSE

In [ ]:
from sklearn.metrics import mean_squared_error

models_list = {
    "Decision Tree" : dt,
    "Random Forest" : rf,
    "XGBoost" : xgb,
    "SVR" : svr,
    "LGBM" : lgbm,
}

for name, model in models_list.items():
    print(name + ": " + str(round(mean_squared_error(model.predict(X_test), y_test, squared = False), 5)))

# Download Objects for API

In [ ]:
import pickle

pickle.dump(encoder, open("cover_crop_encoder.pickle", "wb"))
pickle.dump(rf, open("cover_crop_random_forest.pickle", "wb"))